<table align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/uashogeschoolutrecht/tmoi-ml-19/blob/master/eindopdracht/aanvulling-eindopdracht.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
</table>

In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

We maken een aantal dummy inputs

In [50]:
a = np.random.rand(100,30)
b = np.random.rand(100,30)
c = np.random.rand(100,30)
d = np.random.rand(100, 1)

We kunnen nu datasets maken, met elke shape en vorm die we willen. Als we willen dat de dataset batches ophoest in de vorm `((a, b), (c, d))` dan voeren we dat aan de `tf.data.Dataset`; als we `(X, y)` willen, voeren we dat in; als we `((a, b, c), d)` willen, voeren we dat. Wat je erin stopt, komt er ook uit. Alleen dan gebatched, en geshuffeled, als je dat zou willen.

In [51]:
input = ((a, b), (c, d))
data = tf.data.Dataset.from_tensor_slices(input).batch(32)

In dit geval, krijgen we dus twee tuples, namelijk `(a,b)` en `(c,d)`. We kunnen dat opvangen bijvoorbeeld in twee objecten die we U en V noemen. U wordt dan een tuple, waarvan het eerste item `a` is, en het tweede item `b`, zie hieronder. Een keras model gaat ervan uit dat het eerste ding dat je voert, de input is, en het tweede ding de labels. Dus in dit voorbeeld zouden we het model `(a,b)` voeren als input, en heeft het model schijnbaar twee outputs, twee labels, `(c,d)` die keras gebruikt om de loss mee te berekenen.

In [52]:
for U, V in data.take(1):
  print(U[0].shape)
  print(U[1].shape)
  print(V[0].shape)
  print(V[1].shape)

(32, 30)
(32, 30)
(32, 30)
(32, 1)


Vervolgens kunnen we een functie definieren die een klein model maakt. Dit model neemt maar 1 input, en heeft 1 output. Dit is in feite gewoon een pipeline.

In [54]:
def small_model(shape):
  input = tf.keras.layers.Input(shape)
  output = tf.keras.layers.Dense(100)(input)
  model = tf.keras.models.Model(inputs = [input], outputs=[output], name='matruschka')
  return model

Omdat ons model flexibel is, kunnen we de shape aanpassen elke keer als we het model definieren. Dus laten we een model definieren met `shape=30`.

In [55]:
testmodel = small_model(shape=30)
testmodel.summary()

Model: "matruschka"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_33 (InputLayer)        [(None, 30)]              0         
_________________________________________________________________
dense_10 (Dense)             (None, 100)               3100      
Total params: 3,100
Trainable params: 3,100
Non-trainable params: 0
_________________________________________________________________


En jawel, het werkt. We krijgen een model terug, met als inputshape 30, en een Dense layer van 100 units. We kunnen dat testen met een van onze dummy data `a`, die een shape van 30 had:

In [56]:
output = testmodel(a)
output.shape

TensorShape([100, 100])

Als we nu data invoeren met een andere shape dan gaat het natuurlijk fout.

In [76]:
try:
  testmodel(np.random.rand(100, 10))
except ValueError as e:
  print(str(e))


Input 0 is incompatible with layer matruschka: expected shape=(None, 30), found shape=(100, 10)


Vervolgens gaan we een groot model aanmaken, waarin het kleine model is opgenomen. Op deze manier kun je kleine onderdelen van je model apart definieren, bijvoorbeeld een `encoder` en een `decoder`, of een gedeelte dat de convolutional layers hanteert op een complexe manier. 

In dit voorbeeld kiezen we voor het volgende: we hebben drie inputs, en twee outputs. (Dat gaat dus niet werken met onze datagenerator, want die genereerde twee tuples.)

Die drie inputs gaan alledrie door hetzelfde kleine model, en de outputs kunnen we weer bewerken zoals we willen, bijvoorbeeld concatenaten.

Uiteindelijk maken we een model met de functionele API. In dit geval besluiten we om ook `outc` terug te sturen, gewoon omdat het kan.

Let op, dit is alleen nog maar de definitie: dat is handig, want dan hoeven we niet steeds die code opnieuw op te tuigen. In principe is dit exact hetzelfde als we steeds deden met de hypermodels, behalve dan dat we geen `hp` object invoeren met hyperparameters, maar onze zelfgekozen objecten (in dit voorbeeld alleen maar `shape`, maar je kunt natuurlijk alles wat je wilt daaraan toevoegen).



In [46]:
def big_model(shape):
  a = tf.keras.layers.Input(shape)
  b = tf.keras.layers.Input(shape)
  c = tf.keras.layers.Input(shape)
  small_dense_model = small_model(shape)
  outa = small_dense_model(a)
  outb = small_dense_model(b)
  outc = small_dense_model(c)
  concat = tf.concat([outa, outb, outc], axis=1)
  model = tf.keras.models.Model(inputs = [a,b,c], outputs=[concat, outc])
  return model

Vervolgens kunnen we met deze definitie daadwerkelijk een model aanmaken, met alle parameters die we maar wensen. In dit geval dus een shape van 30.

In [58]:
model = big_model(30)

En jawel, het model ziet eruit zoals we willen. Drie inputs, een matruschka-layer (die dus drie keer wordt hergebruikt) en een concat layer die alles aan elkaar plakt.

In [59]:
model.summary()

Model: "model_12"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_34 (InputLayer)           [(None, 30)]         0                                            
__________________________________________________________________________________________________
input_35 (InputLayer)           [(None, 30)]         0                                            
__________________________________________________________________________________________________
input_36 (InputLayer)           [(None, 30)]         0                                            
__________________________________________________________________________________________________
matruschka (Functional)         (None, 100)          3100        input_34[0][0]                   
                                                                 input_35[0][0]            

Nu kunnen we aan de slag met `.compile` en `.fit` om ons model te trainen.

Nieuwe modellen testen is nu heel simpel:

In [79]:
model2 = big_model(shape = 500)
model2.summary()

Model: "model_15"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_46 (InputLayer)           [(None, 500)]        0                                            
__________________________________________________________________________________________________
input_47 (InputLayer)           [(None, 500)]        0                                            
__________________________________________________________________________________________________
input_48 (InputLayer)           [(None, 500)]        0                                            
__________________________________________________________________________________________________
matruschka (Functional)         (None, 100)          50100       input_46[0][0]                   
                                                                 input_47[0][0]            

We kunnen ook de lagen apart aanroepen

In [62]:
model.layers

En herkennen ook onze matruschka laag.

In [66]:
[x.name for x in model.layers]

['input_34', 'input_35', 'input_36', 'matruschka', 'tf.concat_6']